**Donwload torchtext this versiont**

In [1]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
!pip install torchtext==0.10.1

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import re
import string
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import emoji
import torchtext
import random
import pandas as pd
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
from torchtext.legacy import data
from sklearn.metrics import classification_report

torch.manual_seed(1)

**GPU support available or not**

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


**HyperParameter**

In [76]:
LEARNING_RATE = 0.0003
BATCH_SIZE = 128
NUM_EPOCHS = 15
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 11

**Preparing Data**

In [77]:
df = pd.read_csv('/kaggle/input/extradataset/edos_labelled_aggregated.csv')
df = df[df['label_sexist'] == "sexist"]

# for task A we don't need these things
df.drop(['rewire_id','label_sexist','label_category'], axis=1, inplace=True)

dfTrain=df[df['split']=="train"]
dfVal=df[df['split']=="dev"]
dfTest=df[df['split']=="test"]
#df.drop(['rewire_id','label_category','label_vector'], axis=1, inplace=True)

dfTrain.drop(['split'], axis=1, inplace=True)
dfVal.drop(['split'], axis=1, inplace=True)
dfTest.drop(['split'], axis=1, inplace=True)

print(dfTrain.head())

dfTrain.to_csv('train.csv',index=False)
dfTest.to_csv('test.csv',index=False)
dfVal.to_csv('val.csv',index=False)

                                                 text  \
6   [USER] Leg day is easy. Hot girls who wear min...   
8   I get a new pussy every other week or whenever...   
14  O come on there's no way any men are attracted...   
26  Former BBC journalist Thuto Mali says that uni...   
32  I did. You have to have the bravery to escalat...   

                                         label_vector  
6                 3.3 backhanded gendered compliments  
8   2.3 dehumanising attacks & overt sexual object...  
14  2.3 dehumanising attacks & overt sexual object...  
26  4.2 supporting systemic discrimination against...  
32           1.2 incitement and encouragement of harm  


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [78]:
TEXT = data.Field(
    tokenize='spacy', # default splits on whitespace
    tokenizer_language='en_core_web_sm'
)

### Defining the label processing
LABEL = data.LabelField(dtype=torch.long)

**Loading Data**

In [79]:
fields = [('text', TEXT), ('label_vector', LABEL)]
train_data = data.TabularDataset(path='/kaggle/working/train.csv', format='csv',skip_header=True, fields=fields)

In [80]:
test_data = data.TabularDataset(path='/kaggle/working/test.csv', format='csv',skip_header=True, fields=fields)
valid_data = data.TabularDataset(path='/kaggle/working/val.csv', format='csv',skip_header=True, fields=fields)

In [81]:
print(vars(train_data.examples[0]))

{'text': ['[', 'USER', ']', 'Leg', 'day', 'is', 'easy', '.', 'Hot', 'girls', 'who', 'wear', 'miniskirts', 'get', 'asked', 'out', '.'], 'label_vector': '3.3 backhanded gendered compliments'}


**Data Cleaning**: Not used

In [82]:
class DataCleaning():
    """
        Take a list of strings and preprocess strings, it preforms:
        
    """
    def __init__(self, list_data):
        self.data = list_data
        self.len = len(list_data)

    def clean_data(self):
        self.emojis_to_text()
        self.lowerCase()
        self.linkToTag()
        self.removePunctuations()
        self.removeWordsWithNumber()
        self.rootWord(lemmatizer=True)
        self.removeStopword()
        return self.data
        
    def tokenizer():
        pass
        
    def rootWord(self,lemmatizer=False):
        """
            Steamer is faster than lammatization.
            Steamer cut the last few words, and use the root word
            lemmatizer convert many form a word to the same word. 
            stemmer will retun 'car' for word 'caring'
            while lemmatizer return 'care' for 'caring'
        """
        if not lemmatizer:
            stemmer = PorterStemmer()
            for i in range(self.len):
                self.data[i]=" ".join([stemmer.stem(word) for word in self.data[i].split()])
        else:
            lemmatizer = WordNetLemmatizer()
            for i in range(self.len):
                self.data[i]=" ".join([lemmatizer.lemmatize(word) for word in self.data[i].split()])
        
    def removePunctuations(self):
        removePunc = re.compile(r'[^\w\s]')
        for i in range(self.len):
            self.data[i] = re.sub(removePunc, r" ",self.data[i])

    def emojis_to_text(self):
        """
            Converting image to its text meaning.
            Format: 👍 to ":thumbs_up:"
        """
        for i in range(self.len):
            self.data[i]=emoji.demojize(self.data[i])
            
    def linkToTag(self):
        """replacing web links with '<URL>'"""
        linkRegex = re.compile(r'\b(www|http|https)[^ |\n]*')

        for i in range(self.len):
            self.data[i] = re.sub(linkRegex, r"<URL>",self.data[i])
    
    def removeWordsWithNumber(self):
        """
            Zero or more number of non-whitespace then digit, 
            then Zero or more number of non-whitespace
        """
        wordNumRegex = re.compile(r'\S*\d\S*')
        for i in range(self.len):
            self.data[i]=re.sub(wordNumRegex,r"",self.data[i])
    
    def lowerCase(self):
        """
            It is sometimes important to keep the letter capital 
            as it signifies shouting in form of that word, but it
            depends on use case
        """
        for i in range(self.len):
            self.data[i]=self.data[i].lower()
            
    def removeStopword(self):
        """ Remove stopward from a string"""
        stop = stopwords.words('english')
        for i in range(self.len):
            self.data[i]=' '.join([word for word in self.data[i].split() if word not in stop])

In [83]:
# a=DataCleaning(dfTrain['text'].tolist())
# trainTexts=a.clean_data()
# a=DataCleaning(dfTest['text'].tolist())
# testTexts=a.clean_data()
# a=DataCleaning(dfVal['text'].tolist())
# valTexts=a.clean_data()

In [84]:
# trainText=dfTrain['text'].tolist()
# testText=dfTest['text'].tolist()
# valText=dfVal['text'].tolist()

In [85]:
len(train_data)

3398

**Build vocab**

In [86]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 10905
Number of classes: 11


In [87]:
print(LABEL.vocab.stoi)

defaultdict(None, {'2.1 descriptive attacks': 0, '2.2 aggressive and emotive attacks': 1, '3.1 casual use of gendered slurs, profanities, and insults': 2, '3.2 immutable gender differences and gender stereotypes': 3, '4.2 supporting systemic discrimination against women as a group': 4, '1.2 incitement and encouragement of harm': 5, '2.3 dehumanising attacks & overt sexual objectification': 6, '4.1 supporting mistreatment of individual women': 7, '3.3 backhanded gendered compliments': 8, '1.1 threats of harm': 9, '3.4 condescending explanations or unwelcome advice': 10})


**Dataloader**

In [88]:
train_loader, valid_loader, test_loader = \
    data.BucketIterator.splits(
        (train_data, valid_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.text),
         device=device
    )

**Shape of data**

In [89]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.text.size()}')
    print(f'Target vector size: {batch.label_vector.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.text.size()}')
    print(f'Target vector size: {batch.label_vector.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.text.size()}')
    print(f'Target vector size: {batch.label_vector.size()}')
    break

Train
Text matrix size: torch.Size([56, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([19, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([13, 128])
Target vector size: torch.Size([128])


**Module**

In [90]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        #self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)        
        
        self.fc1 = torch.nn.Linear(hidden_dim, hidden_dim//2)
        self.fc2 = torch.nn.Linear(hidden_dim//2, output_dim)
        

    def forward(self, text):
        # text dim: [sentence length, batch size]
        
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]
        
        output = self.fc1(hidden)
        output = self.fc2(F.relu(output))
        return output

In [91]:
model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # could use 1 for binary classification
)

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

**Accuracy Report**

In [92]:
def compute_accuracy(model, data_loader, device,report=False):

    predict=[]
    actual=[]
    
    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
            
            predict+=predicted_labels.tolist()                         
            actual+=targets.tolist()
            
        if report:    
            print(classification_report(actual, predict, labels=[0,1]))
            
    return correct_pred.float()/num_examples * 100

In [93]:
DEVICE=device

**Training**

In [95]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.text.to(device)
        labels = batch_data.label_vector.to(device)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    torch.save(model.state_dict(), 'lstm'+str(epoch)+".pt")
            
    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, device):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, device):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, device):.2f}%')

Epoch: 001/015 | Batch 000/027 | Loss: 2.3987
training accuracy: 21.13%
valid accuracy: 21.19%
Time elapsed: 0.01 min
Epoch: 002/015 | Batch 000/027 | Loss: 2.1050
training accuracy: 18.72%
valid accuracy: 18.93%
Time elapsed: 0.02 min
Epoch: 003/015 | Batch 000/027 | Loss: 2.0951
training accuracy: 21.25%
valid accuracy: 21.19%
Time elapsed: 0.02 min
Epoch: 004/015 | Batch 000/027 | Loss: 2.0531
training accuracy: 21.31%
valid accuracy: 20.78%
Time elapsed: 0.03 min
Epoch: 005/015 | Batch 000/027 | Loss: 2.0617
training accuracy: 21.31%
valid accuracy: 20.78%
Time elapsed: 0.04 min
Epoch: 006/015 | Batch 000/027 | Loss: 2.0043
training accuracy: 20.04%
valid accuracy: 19.55%
Time elapsed: 0.05 min
Epoch: 007/015 | Batch 000/027 | Loss: 1.9484
training accuracy: 21.22%
valid accuracy: 20.99%
Time elapsed: 0.05 min
Epoch: 008/015 | Batch 000/027 | Loss: 2.1134
training accuracy: 21.28%
valid accuracy: 20.78%
Time elapsed: 0.06 min
Epoch: 009/015 | Batch 000/027 | Loss: 2.0921
training a

**Loading the best Model**

In [96]:
path = "/kaggle/working/lstm14.pt"
model.load_state_dict(torch.load(path))

<All keys matched successfully>

**F1 score at the test data**

In [97]:
with torch.set_grad_enabled(False):
    print('test accuracy: ',compute_accuracy(model, test_loader, device,True))

              precision    recall  f1-score   support

           0       0.21      0.79      0.34       205
           1       0.00      0.00      0.00       192

   micro avg       0.21      0.41      0.28       397
   macro avg       0.11      0.39      0.17       397
weighted avg       0.11      0.41      0.17       397

test accuracy:  tensor(21.4433, device='cuda:0')


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [98]:
with torch.set_grad_enabled(False):
    print('test accuracy: ',compute_accuracy(model, valid_loader, device,True))

              precision    recall  f1-score   support

           0       0.21      0.92      0.35       102
           1       0.00      0.00      0.00        96

   micro avg       0.21      0.47      0.30       198
   macro avg       0.11      0.46      0.17       198
weighted avg       0.11      0.47      0.18       198

test accuracy:  tensor(21.8107, device='cuda:0')


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Macro F1 score is 0.17

Reference

1. https://www.youtube.com/watch?v=CrS-LFXEiyk
2. https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
3. https://github.com/rasbt/stat453-deep-learning-ss21/blob/main/L15/1_lstm.ipynb